In [24]:
using JuMP, Gurobi

In [27]:
origin = zeros(Int32, 12, 12)
origin[3:2:10, 3:2:10] .= 1
origin[4:2:10, 4:2:10] .= 1

destination = zeros(Int32, 12, 12)
destination[3:2:10, 4:2:11] .= 1
destination[4:2:10, 5:2:11] .= 1
;

In [68]:
function getDistance(origin, destination, ws, epsilon)
    image_width, image_height = size(origin)
    xIter, yIter, fIter = 0:(image_width-1), 0:(image_height-1), 0:(2*ws)
    
    m = Model(Gurobi.Optimizer)
    @variable(m, v[xIter, yIter, fIter, fIter])
    
    # add constraints that if a pixel shift is outside image, it must be 0
    isOutside(x, xs, h) = x + (xs - ws) < 0 || x + (xs - ws) > h - 1
    for x in xIter, y in yIter, xs in fIter, ys in fIter
        if  isOutside(x, xs, image_width) || isOutside(y, ys, image_height)
            @constraint(m, v[x, y, xs, ys] == 0)
        end
    end
    
    # constraints that all source pixels must shift in at least 1 direction
    @constraint(m, mustShiftC[x in xIter, y in yIter], sum(v[x,y,:,:]) >= 1)
    
    
    # constraints that all dest pixels must be mapped to
    isInside(x, xs, h) = x - (xs - ws) >= 0 && x - (xs - ws) < h
    @constraint(m, ontoC[x in xIter, y in yIter],
        sum(v[x - (xs - ws), y - (ys - ws), xs, ys] for xs in fIter, ys in fIter
                if isInside(x, xs, image_width) && isInside(y, ys, image_height)
            ) >= 1
    )
    
    return m
end

getDistance (generic function with 1 method)

In [69]:
m = getDistance(origin, destination, 1, 0.1)
println(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-02
Feasibility
Subject to
 v[0,0,0,0] = 0.0
 v[0,0,0,1] = 0.0
 v[0,0,0,2] = 0.0
 v[0,0,1,0] = 0.0
 v[0,0,2,0] = 0.0
 v[0,1,0,0] = 0.0
 v[0,1,0,1] = 0.0
 v[0,1,0,2] = 0.0
 v[0,2,0,0] = 0.0
 v[0,2,0,1] = 0.0
 v[0,2,0,2] = 0.0
 v[0,3,0,0] = 0.0
 v[0,3,0,1] = 0.0
 v[0,3,0,2] = 0.0
 v[0,4,0,0] = 0.0
 v[0,4,0,1] = 0.0
 v[0,4,0,2] = 0.0
 v[0,5,0,0] = 0.0
 v[0,5,0,1] = 0.0
 v[0,5,0,2] = 0.0
 v[0,6,0,0] = 0.0
 v[0,6,0,1] = 0.0
 v[0,6,0,2] = 0.0
 v[0,7,0,0] = 0.0
 v[0,7,0,1] = 0.0
 v[0,7,0,2] = 0.0
 v[0,8,0,0] = 0.0
 v[0,8,0,1] = 0.0
 v[0,8,0,2] = 0.0
 v[0,9,0,0] = 0.0
 v[0,9,0,1] = 0.0
 v[0,9,0,2] = 0.0
 v[0,10,0,0] = 0.0
 v[0,10,0,1] = 0.0
 v[0,10,0,2] = 0.0
 v[0,11,0,0] = 0.0
 v[0,11,0,1] = 0.0
 v[0,11,0,2] = 0.0
 v[0,11,1,2] = 0.0
 v[0,11,2,2] = 0.0
 v[1,0,0,0] = 0.0
 v[1,0,1,0] = 0.0
 v[1,0,2,0] = 0.0
 v[1,11,0,2] = 0.0
 v[1,11,1,2] = 0.0
 v[1,11,2,2] = 0.0
 v[2,0,0,0] = 0.0
 v[2,0,1,0] = 0.0
 v[2,0,2,0]